In [1]:
import gym
from gym import spaces
import numpy as np
import pygame

import scipy
import gdown
import h5py
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#!git clone https://github.com/ziatdinovmax/im2spec.git
%cd im2spec

/Users/adivatsavai/Adi/Curiosity RL/im2spec


In [3]:
import os
from copy import deepcopy as dc

import matplotlib.gridspec as gridspec



import torch
import torch.nn as nn
import torch.nn.functional as F

from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import decomposition
from sklearn.mixture import GaussianMixture
from skimage.measure import compare_ssim as ssim

from im2spec.models import im2spec, spec2im
from im2spec.utils import create_training_set, predict, loop_area, encode, decode, latent_gmm
from im2spec.train_utils import trainer

In [4]:
#!gdown https://drive.google.com/uc?id=1yrGXukGspctsSYrHn3grY_isQR2VFPLd

In [5]:
h5 = h5py.File(r'BEPS_kyleSample_3um_0018.h5', 'r+')

spec_val = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_001/Spectroscopic_Values']


amp = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Amplitude [V]'].reshape(100,100,384)
pha = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Phase [rad]'].reshape(100,100,384)
fre = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Frequency [Hz]'].reshape(100,100,384)
q = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Quality Factor'].reshape(100,100,384)
#r2 = h5['Measurement_000/Channel_001/Raw_Data-SHO_Fit_000/Guess']['R2 Criterion'].reshape(100,100,384)
pola = (amp*np.cos(pha+2))*1E5


In [17]:
class environment(gym.Env):
    def __init__(self, image, spectra, start = [50, 50], image_patch = 5):
        super(environment, self).__init__()
        self.image_patch = image_patch
        self.radius = int((image_patch - 1)/2)+1
        self.image = image
        self.spectra = spectra
        self.color = 255/(image.max() - image.min()) * image - 255/(image.max() - image.min()) * image.min()
        self.pos = start
        self.action_space = spaces.Discrete(4)
        self.num_rows = 100
        self.num_columns = 100
        self.pos_X = []
        self.X = []
        self.y = []
        
        for i in range(-1, 2):
            for j in range(-1, 2):
                self.pos_X.append([start[0]+i, start[1]+j])
        
        for ind in self.pos_X:
            self.X.append(image[ind[0]-int((image_patch-1)/2): ind[0] + int((image_patch+1)/2), ind[1]-int((image_patch-1)/2): ind[1] + int((image_patch+1)/2)])
            self.y.append(self.spectra[ind[0], ind[1]])
            
        
        
        
        self.been = np.zeros([self.num_rows, self.num_columns])
        self.seen = np.zeros([self.num_rows, self.num_columns])
        self.been[self.pos[0], self.pos[1]] = 1
        self.seen[self.pos[0], self.pos[1]] = 1
        
        for i in range(self.radius + 1):
            for j in range(self.radius + 1):
                self.seen[self.pos[0]+i, self.pos[1]+j] = 1
                self.seen[self.pos[0]-i, self.pos[1]+j] = 1
                self.seen[self.pos[0]+i, self.pos[1]-j] = 1
                self.seen[self.pos[0]-i, self.pos[1]-j] = 1
        
        self.observation_space = spaces.Tuple((spaces.Discrete(self.num_rows), spaces.Discrete(self.num_columns)))
        
        pygame.init()
        self.cell_size = 10
        
        self.screen = pygame.display.set_mode((self.num_columns * self.cell_size, self.num_rows * self.cell_size))
        
    
    def step(self, action):
        
        #self.seen[self.pos[0], self.pos[1]] = 0
        
        new_pos = self.pos
        if action == 0:  
            new_pos[0] += -1
            new_pos[1] += -1
        elif action == 1: 
            new_pos[0] += -1
            new_pos[1] += 0
        elif action == 2:  
            new_pos[0] += -1
            new_pos[1] += 1
        elif action == 3: 
            new_pos[0] += 0
            new_pos[1] += -1
        elif action == 4:
            new_pos[0] += 0
            new_pos[1] += 1
        elif action == 5:
            new_pos[0] += 1
            new_pos[1] += -1
        elif action == 6:
            new_pos[0] += 1
            new_pos[1] += 0
        elif action == 7:
            new_pos[0] += 1
            new_pos[1] += 1
            
        
        if new_pos[1] >= self.radius+1 and new_pos[1] < self.num_columns - self.radius-1 and new_pos[0] >= self.radius+1 and new_pos[0] < self.num_rows - self.radius-1:
            

            self.pos = new_pos
            self.been[self.pos[0], self.pos[1]] = 1
            
            for i in range(self.radius + 1):
                for j in range(self.radius + 1):
                    self.seen[self.pos[0]+i, self.pos[1]+j] = 1
                    self.seen[self.pos[0]-i, self.pos[1]+j] = 1
                    self.seen[self.pos[0]+i, self.pos[1]-j] = 1
                    self.seen[self.pos[0]-i, self.pos[1]-j] = 1
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if not [self.pos[0]+i, self.pos[1]+j] in self.pos_X:
                        self.pos_X.append([self.pos[0]+i, self.pos[1]+j])
                        ind = [self.pos[0]+i, self.pos[1]+j]
                        self.X.append(self.image[ind[0]-int((self.image_patch-1)/2): ind[0] + int((self.image_patch+1)/2), ind[1]-int((self.image_patch-1)/2): ind[1] + int((self.image_patch+1)/2)])
                        
                        self.y.append(spectra[ind[0], ind[1]])
                    
            
            
        
        
        
    
    def state(self):
        return(self.pos, self.been, self.seen, self.pos_X, np.array(self.X).reshape([len(self.X), 1, self.image_patch, self.image_patch]), np.array(self.y).reshape([len(self.y), 1, 64]))
    
    def render(self):

        self.screen.fill((255, 255, 255)) 
        
        for row in range(self.num_rows):
            for col in range(self.num_columns):
                cell_left = col * self.cell_size
                cell_top = row * self.cell_size
                
                if self.seen[row, col] == 1 and not (self.pos[0] == row and self.pos[1] == col):
                    pygame.draw.rect(self.screen, (0, self.color[row, col], 0), (cell_left, cell_top, self.cell_size, self.cell_size))
                
                elif self.pos[0] == row and self.pos[1] == col:
                    pygame.draw.rect(self.screen, (255, 0, 0), (cell_left, cell_top, self.cell_size, self.cell_size))
                else:
                    pygame.draw.rect(self.screen, (0, 0, self.color[row, col]), (cell_left, cell_top, self.cell_size, self.cell_size))
                

        pygame.display.update()

            
    def reset(self, start = [50, 50]):
        self.pos = start
        self.pos_X = []
        self.X = []
        self.y = []
        
    
            
    
    

In [18]:
class agent():
    
    def __init__(self, image_patch):
        self.model = im2spec((image_patch, image_patch), 64, 10)
    
    def train(self, X, y):
        self.model = trainer(self.model, X, y, X, y, num_epochs=1, savename="im2spec_lv{}".format(10)).run()
        
        
    def move(self, image, spectra, been, pos):

        self.pos = pos
        cand = [[self.pos[0]-1, self.pos[1]-1], [self.pos[0]-1, self.pos[1]], [self.pos[0]-1, self.pos[1]+1],
               [self.pos[0], self.pos[1]-1], [self.pos[0], self.pos[1]+1], 
               [self.pos[0]+1, self.pos[1]-1], [self.pos[0]+1, self.pos[1]], [self.pos[0]+1, self.pos[1]+1]]
        truth_image = []
        truth_spectra = []
        for ind in cand:
            truth_image.append(image[ind[0]-int((image_patch-1)/2): ind[0] + int((image_patch+1)/2), ind[1]-int((image_patch-1)/2): ind[1] + int((image_patch+1)/2)])
            truth_spectra.append(spectra[ind[0], ind[1]])
        
        truth_image = np.array(truth_image).reshape([len(truth_image), 1, image_patch, image_patch])
        truth_spectra = np.array(truth_spectra).reshape([len(truth_spectra), 1, 64])

        
        
        pred_loop = predict(self.model, truth_image)
        
        i_reward = []
        
        for i in range(len(cand)):
            
            if been[cand[i][0], cand[i][1]] == 1:
                
                i_reward.append((np.sum((pred_loop[i] - truth_spectra[i])**2))/5)
                
            else:
                i_reward.append(np.sum((pred_loop[i] - truth_spectra[i])**2))

        return(np.array(i_reward).argmax())
        
        
        
        
        
                    
                    
        
                    
        
        
        
        
        
        
    
    
        

In [20]:
image_patch = 5
image, spectra = pola[:,:,0], pola[:,:,::2][:,:,0:64]

env = environment(image, spectra, start = [50, 50], image_patch = image_patch)

ag = agent(env.image_patch)

env.render()
pos, been, seen, pos_X, X, y = env.state()

ag.train(np.array(X), np.array(y).reshape([len(pos_X), 1, 64]))
action = ag.move(image, spectra, been, pos)



for i in range(250):
    pygame.event.get()
    obs = env.step(action)
    env.render()
    pos, been, seen, pos_X, X, y = env.state()
    
    ag.train(X, y)
    action = ag.move(image, spectra, been, pos)

env.reset()

Epoch: 1... Training loss: 54.92897... Test loss: 54.82659
Epoch: 1... Training loss: 43.60955... Test loss: 43.61004
Epoch: 1... Training loss: 36.65872... Test loss: 37.12713
Epoch: 1... Training loss: 31.5981... Test loss: 32.33418
Epoch: 1... Training loss: 27.43365... Test loss: 28.23935
Epoch: 1... Training loss: 24.86299... Test loss: 25.62246
Epoch: 1... Training loss: 23.28187... Test loss: 24.04193
Epoch: 1... Training loss: 21.07909... Test loss: 21.70276
Epoch: 1... Training loss: 18.89764... Test loss: 19.45826
Epoch: 1... Training loss: 17.51886... Test loss: 18.01532
Epoch: 1... Training loss: 16.37806... Test loss: 16.81385
Epoch: 1... Training loss: 15.10414... Test loss: 15.50041
Epoch: 1... Training loss: 14.4863... Test loss: 14.82085
Epoch: 1... Training loss: 13.34858... Test loss: 13.69378
Epoch: 1... Training loss: 13.03666... Test loss: 13.30212
Epoch: 1... Training loss: 12.42921... Test loss: 12.72047
Epoch: 1... Training loss: 12.23759... Test loss: 12.40957